Install libraries

In [1]:
!pip install sentinelsat
!pip install geopandas
!pip install rasterio
!pip install pycrs

     |████████████████████████████████| 931kB 9.2MB/s 
     |████████████████████████████████| 14.7MB 305kB/s 
     |████████████████████████████████| 10.4MB 57.2MB/s 
     |████████████████████████████████| 18.1MB 243kB/s 
  Created wheel for pycrs: filename=PyCRS-1.0.2-cp36-none-any.whl size=32683 sha256=e9847d2b699e41cb0df54b473fba5218cd28227eb90e9f0c076d73fd3b055709
  Stored in directory: /root/.cache/pip/wheels/1a/c9/56/648f762bd41fdb03454be51b8a7069795925168833cb61f316
Successfully built pycrs


Import libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from sentinelsat import SentinelAPI
import folium
import shapely
from shapely.geometry import MultiPolygon
from shapely.geometry import Polygon
from shapely.geometry import shape
from shapely.geometry import box
import urllib.request as request
import json
from datetime import datetime
import zipfile
import rasterio as rio
from rasterio.plot import show
from rasterio.mask import mask
import os
from fiona.crs import from_epsg
from skimage import exposure
from PIL import Image
import matplotlib
import pycrs
import warnings

warnings.filterwarnings('ignore')

Extract Geometries

In [0]:
def extract_geometries(max_records):
  current_record = 0
  names = []
  countries = []
  latitudes = []
  longitudes = []
  geometries = []
  while len(names) < max_records:
    water_url = f'https://water.blue-dot-observatory.com/api/waterbodies/{current_record}/index.html'
    try:
      water_body_data = json.loads(request.urlopen(water_url).read().decode())
      names.append(water_body_data['properties']['name'])
      countries.append(water_body_data['properties']['country'])
      latitudes.append(water_body_data['properties']['lat'])
      longitudes.append(water_body_data['properties']['long'])
      geometries.append(water_body_data['nominal_outline']['geometry'])
    except:
      pass
    current_record += 1
      
  #Create Dataframe
  water_bodies = pd.DataFrame({'Name':names,
                             'Country':countries,
                             'Latitude':latitudes,
                             'Longitude':longitudes,
                             'Geometry':geometries})
  
  return water_bodies

In [0]:
water_bodies = extract_geometries(10)

In [43]:
water_bodies

,Name,Country,Latitude,Longitude,Geometry
0,Waduk Saguling,Indonesia,-6.921428,107.432124,"{'coordinates': [[[[107.3995006, -6.9632461], ..."
1,Karangkates,Indonesia,-8.182967,112.481540,"{'coordinates': [[[112.446266, -8.1639212], [1..."
2,Waduk Wadaslintang,Indonesia,-7.578089,109.784388,"{'coordinates': [[[109.7529273, -7.5696733], [..."
3,Togo Ngebe,Indonesia,-7.797072,111.632870,"{'coordinates': [[[111.6267164, -7.795229], [1..."
4,Cirata,Indonesia,-6.737726,107.298976,"{'coordinates': [[[107.24025, -6.716], [107.24..."
5,Riam Kanan,Indonesia,-3.528408,115.077082,"{'coordinates': [[[115.0013237, -3.5811811], [..."
6,Gondang,Indonesia,-7.209774,112.269189,"{'coordinates': [[[112.2567124, -7.2111565], [..."
7,Waduk Nglahor,Indonesia,-8.143460,112.468104,"{'coordinates': [[[112.4513062, -8.1452087], [..."
8,Waduk Kedung Ombo,Indonesia,-7.284976,110.811855,"{'coordinates': [[[110.753302, -7.3012137], [1..."
9,Waduk Selorejo,Indonesia,-7.864118,112.365434,"{'coordinates': [[[112.3549104, -7.8722693], [..."


Read login credentials

In [0]:
with open('/content/drive/My Drive/Sentinel Hub Credentials/login_info.txt') as file:
  credentials = file.readlines()
  credentials = [credential.strip() for credential in credentials]

Download Satellite Images

In [0]:
api = SentinelAPI(credentials[0], credentials[1], 'https://scihub.copernicus.eu/dhus')

In [0]:
today = datetime.today().strftime('%Y%m%d')

In [0]:
water_body_geometry = water_bodies.iloc[0]['Geometry']

In [0]:
water_body_shape = shape(water_body_geometry)

In [0]:
shape_simplified = box(water_body_shape.bounds[0], water_body_shape.bounds[1], water_body_shape.bounds[2], water_body_shape.bounds[3])

In [15]:
results = api.query(shape_simplified, date = ('20140101', today), platformname = 'Sentinel-2', processinglevel = 'Level-1C', cloudcoverpercentage = (0,5))
results_gdf = api.to_geodataframe(results).sort_values(['cloudcoverpercentage'], ascending=True)
uuid = results_gdf.iloc[0]['uuid']
filename = '/content/' + results_gdf.iloc[0]['title'] + '.zip'
data = api.download(uuid)

Querying products: 100%|██████████| 144/144 [00:01<00:00, 109.88 products/s]
Downloading: 100%|██████████| 869M/869M [00:29<00:00, 29.7MB/s]
MD5 checksumming: 100%|██████████| 869M/869M [00:01<00:00, 458MB/s]


In [0]:
with zipfile.ZipFile(filename, 'r') as zip_ref:
  zip_ref.extractall('/content/drive/My Drive/Satellite Data/')
folder = os.listdir('/content/drive/My Drive/Satellite Data/' + data['title'] + '.SAFE/GRANULE/')[0]
directory = '/content/drive/My Drive/Satellite Data/' + data['title'] + '.SAFE/GRANULE/' + folder + '/IMG_DATA/'

In [0]:
os.remove(filename)

In [0]:
for f in os.listdir(directory):
  if '_B02' in f:
    band_2 = rio.open(directory + f)
  elif '_B03' in f:
    band_3 = rio.open(directory + f)
  elif '_B04' in f:
    band_4 = rio.open(directory + f)

In [0]:
with rio.open('RGB.tiff','w',driver='Gtiff', width=band_4.width, height=band_4.height, 
              count=3,crs=band_4.crs,transform=band_4.transform, dtype=band_4.dtypes[0]) as rgb:
    rgb.write(band_4.read(1),1) 
    rgb.write(band_3.read(1),2) 
    rgb.write(band_2.read(1),3) 
    rgb.close()

In [0]:
minx, miny = -98.2571703, 30.3577993
maxx, maxy = -97.8719835, 30.5810474
bbox = box(minx, miny, maxx, maxy)

In [0]:
geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326))

In [0]:
src = rio.open('/content/RGB.tiff')

In [0]:
geo = geo.to_crs(crs=src.crs.data)

In [0]:
def get_features(gdf):
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [31]:
coordinates = get_features(geo)
print(coordinates)

[{'type': 'Polygon', 'coordinates': [[[608405.6357454355, 3358972.860974127], [608158.6021092285, 3383714.6797456937], [571224.129600805, 3383407.870148308], [571386.7872278213, 3358667.38035143], [608405.6357454355, 3358972.860974127]]]}]


In [0]:
out_img, out_transform = mask(dataset=src, shapes=coordinates, crop=True)

In [0]:
out_meta = src.meta.copy()

In [0]:
epsg_code = int(src.crs.data['init'][5:])

In [0]:
out_meta.update({"driver": "GTiff",
                 "height": out_img.shape[1],
                 "width": out_img.shape[2],
                 "transform": out_transform,
                 "crs": pycrs.parse.from_epsg_code(epsg_code).to_proj4()
                 }
                         )

In [0]:
with rio.open('/content/output.tiff', "w", **out_meta) as dest:
        dest.write(out_img)

In [0]:
img = rio.open('output.tiff')
image = np.array([img.read(1), img.read(2), img.read(3)]).transpose(1,2,0)
p2, p98 = np.percentile(image, (2,98))
image = exposure.rescale_intensity(image, in_range=(p2, p98)) / 100000

In [0]:
matplotlib.image.imsave('lake_travis.jpg', image)

In [0]:
os.remove('/content/RGB.tiff')
os.remove('/content/output.tiff')

In [0]:
def create_mask(i):
  #Image
  filename = data.iloc[i][0]
  image_1 = Image.open(directory + '/' + filename)

  #Polygon
  image_2 = image_1.copy()
  draw = ImageDraw.Draw(image_2)
  draw.rectangle([(0,0),image_1.size], fill='black')

  for shape in range(len(data['regions'][i])):
    x = data['regions'][i][str(shape)]['shape_attributes']['all_points_x']
    y = data['regions'][i][str(shape)]['shape_attributes']['all_points_y']
    coordinates = list(zip(x,y))
    draw.polygon(coordinates, fill='white')

  #Merge images
  final_image = Image.blend(image_1, image_2, alpha=1)
  return final_image